In [1]:
def query_generator(county_fips, time):
    query = ("""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX covid-obs-property: <http://covid.geog.ucsb.edu/lod/observedproperty/>
PREFIX covid-instant: <http://covid.geog.ucsb.edu/lod/instant/>
PREFIX covid-place: <http://covid.geog.ucsb.edu/lod/place/>
PREFIX covid: <http://covid.geog.ucsb.edu/lod/ontology/>
PREFIX covid-method: <http://covid.geog.ucsb.edu/lod/method/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>


select ?modelName (min(abs(?predict_value-?groundtruth_value)) as ?min_gap) where {
     
  ?target a covid:Target ;
            sosa:observedProperty covid-obs-property:cum_death ;
            sosa:phenomenonTime ?time ;
            covid:point ?predict_value ;
            sosa:hasFeatureOfInterest ?place ;
            ^sosa:hasMember/^sosa:hasMember/sosa:madeBySensor ?model ;
     .
   
  ?model rdfs:label ?modelName .
            
  ?ground_truth a covid:GroundTruth ;
              sosa:hasFeatureOfInterest ?place ;
              sosa:observedProperty covid-obs-property:cum_death ;
              sosa:phenomenonTime ?time ;
              covid:point ?groundtruth_value .
  
  ?place covid:placeFIPS '%s' .
    
  ?time time:inXSDDateTime '%s' .
}
group by ?modelName
order by ?min_gap
""")% (county_fips, time)
    return query


### This function organizes the queries results from query_generator(model_url, date_str)

def extract_results(result):
    result_list = []
    
    for item in result['results']['bindings']:
        result_list.append([item['modelName']['value'], item['min_gap']['value']])
    
    return result_list

import pandas as pd
state_fips = pd.read_csv("../Question7-Analysis/fips-codes/state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))

# Map for the method of JHUAPL-Bucky

In [31]:
import pandas as pd
from pymantic import sparql
import csv
import plotly.express as px
import csv

fips_state_list = state_fips['fips'].tolist()
result_out = []
for item in fips_state_list:
    print("processing county:%s"%item)    
    query_item = query_generator(item, "2022-02-05")
    server = sparql.SPARQLServer('http://stko-roy.geog.ucsb.edu/covid/repositories/COVID-Forecast-Graph')
    result = server.query(query_item)
    result_list = extract_results(result)
    result_pd = pd.DataFrame(result_list, columns =['Model', 'Error'])  
    if len(result_pd.index[result_pd['Model'] == 'JHUAPL-Bucky'].tolist()):
    
        index = result_pd.index[result_pd['Model'] == 'JHUAPL-Bucky'].tolist()[0]
        ratio = (index+1)/len(result_pd)  # index start from 0, so +1
    else:
        index = 'NA'
        ratio = 'NA'
        print("county %s has no ranking"%item)
    result_out.append([item, index+1, len(result_pd), ratio])

processing county:01
processing county:02
processing county:04
processing county:05
processing county:06
processing county:08
processing county:09
processing county:10
processing county:12
processing county:13
processing county:15
processing county:16
processing county:17
processing county:18
processing county:19
processing county:20
processing county:21
processing county:22
processing county:23
processing county:24
processing county:25
processing county:26
processing county:27
processing county:28
processing county:29
processing county:30
processing county:31
processing county:32
processing county:33
processing county:34
processing county:35
processing county:36
processing county:37
processing county:38
processing county:39
processing county:40
processing county:41
processing county:42
processing county:44
processing county:45
processing county:46
processing county:47
processing county:48
processing county:49
processing county:50
processing county:51
processing county:53
processing co

In [32]:
result_out_pd = pd.DataFrame(result_out, columns =['state_fips', 'rank', 'total', 'ratio']) 
result_out_pd

,state_fips,rank,total,ratio
0,01,6,35,0.171429
1,02,17,34,0.500000
2,04,12,35,0.342857
3,05,12,35,0.342857
4,06,28,35,0.800000
5,08,15,35,0.428571
6,09,7,35,0.200000
7,10,21,36,0.583333
8,12,13,36,0.361111
9,13,26,36,0.722222


In [33]:
def convert2class(value):
    if value <=0.1:
        return "Top 10%"
    elif value>0.1 and value<=0.3:
        return "Top 30%"
    elif value>0.3 and value <=0.5:
        return "Top 50%"
    elif value>0.5 and value <=0.7:
        return "Top 70%"
    #elif value>0.7 and value <=0.9:
    #    return "Top 90%"
    #elif value>0. and value <=0.9:
    #    return "Top 80%"
    else:
        return "Top 90%"

In [34]:
## process the data and visualize it
result_out_pd['Class'] = result_out_pd['ratio'].apply(lambda row: convert2class(row))
import pandas as pd
state_fips = pd.read_csv("state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))


state_fips_merged = state_fips.merge(result_out_pd, left_on='fips', right_on='state_fips')

state_fips_merged_sorted = state_fips_merged.sort_values(by=['Class'])

fig = px.choropleth(state_fips_merged_sorted, locations='state_abbr', locationmode="USA-states", color='Class', 
                    scope="usa",
                   #color_discrete_sequence = px.colors.qualitative.Light24
                   color_discrete_sequence = ['green', 'lightgreen', 'yellow', 'gold', '#FD3216','' ],
                   )

#fig.update_layout(
#    title_text = 'Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Cum Death on Jan 9 2021?'
#)
#fig.show()
fig.write_image("JHUAPL-Bucky_02052022.png")

In [35]:
result_out_pd

,state_fips,rank,total,ratio,Class
0,01,6,35,0.171429,Top 30%
1,02,17,34,0.500000,Top 50%
2,04,12,35,0.342857,Top 50%
3,05,12,35,0.342857,Top 50%
4,06,28,35,0.800000,Top 90%
5,08,15,35,0.428571,Top 50%
6,09,7,35,0.200000,Top 30%
7,10,21,36,0.583333,Top 70%
8,12,13,36,0.361111,Top 50%
9,13,26,36,0.722222,Top 90%
